In [37]:
import requests
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

from secrets import *

In [38]:
dbname = 'cluj'
engine = create_engine('postgres://%s:%s@localhost/%s'%('docker','docker',dbname))

if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


In [2]:
cookies = {"swid": AUTH['swid'],
           "espn_s2": AUTH['espn_s2']}

In [3]:
league_id = 84057

In [4]:
url = 'https://fantasy.espn.com/apis/v3/games/FBA/seasons/2020/segments/0/leagues/' + str(league_id)
params={"view": "mBoxscore"}
r = requests.get(url, params=params, cookies=cookies)
data = r.json()

In [5]:
data.keys()

dict_keys(['draftDetail', 'gameId', 'id', 'schedule', 'scoringPeriodId', 'seasonId', 'segmentId', 'settings', 'status', 'teams'])

In [39]:
# could use this to write info to database about who is what team/names, etc
#data['teams']

In [ ]:
# I need to put opponent 

In [43]:
for matchup in data['schedule']:
    
    matchup_period = matchup['matchupPeriodId']
    print(matchup_period)
    
    if matchup_period == 17: # current matchup, do this manually for now, in the future I can pull just matchup that finished
        break
    
    for roles in [{'team_role':'home', 'opponent_role':'away'}, {'team_role':'away', 'opponent_role':'home'}]:
    
        team_role = roles['team_role']
        opponent_role = roles['opponent_role']
        
        team = matchup[team_role]['teamId']
        opponent = matchup[opponent_role]['teamId']
        print(team, opponent)

        pts = matchup[team_role]['cumulativeScore']['scoreByStat']['0']['score']
        blks = matchup[team_role]['cumulativeScore']['scoreByStat']['1']['score']
        threes = matchup[team_role]['cumulativeScore']['scoreByStat']['17']['score']
        stls = matchup[team_role]['cumulativeScore']['scoreByStat']['2']['score']
        asts = matchup[team_role]['cumulativeScore']['scoreByStat']['3']['score']
        fg_pct = matchup[team_role]['cumulativeScore']['scoreByStat']['19']['score']
        ft_pct = matchup[team_role]['cumulativeScore']['scoreByStat']['20']['score']
        rebs = matchup[team_role]['cumulativeScore']['scoreByStat']['6']['score']

        opp_pts = matchup[opponent_role]['cumulativeScore']['scoreByStat']['0']['score']
        opp_blks = matchup[opponent_role]['cumulativeScore']['scoreByStat']['1']['score']
        opp_threes = matchup[opponent_role]['cumulativeScore']['scoreByStat']['17']['score']
        opp_stls = matchup[opponent_role]['cumulativeScore']['scoreByStat']['2']['score']
        opp_asts = matchup[opponent_role]['cumulativeScore']['scoreByStat']['3']['score']
        opp_fg_pct = matchup[opponent_role]['cumulativeScore']['scoreByStat']['19']['score']
        opp_ft_pct = matchup[opponent_role]['cumulativeScore']['scoreByStat']['20']['score']
        opp_rebs = matchup[opponent_role]['cumulativeScore']['scoreByStat']['6']['score']

        data_dict = {'matchup_period':matchup_period, 'team':team, 'opponent':opponent,
                     'pts':pts, 'blks':blks, 'threes':threes, 'stls':stls, 
                     'asts':asts, 'fg_pct':fg_pct, 'ft_pct':ft_pct, 'rebs':rebs,
                     'opp_pts':opp_pts, 'opp_blks':opp_blks, 'opp_threes':opp_threes, 'opp_stls':opp_stls, 
                     'opp_asts':opp_asts, 'opp_fg_pct':opp_fg_pct, 'opp_ft_pct':opp_ft_pct, 'opp_rebs':opp_rebs
                    }
        df = pd.DataFrame(data_dict, index=[0])

        df.to_sql('matchup_results', con=engine, if_exists='append', index=False)
        

1
12 4
4 12
1
2 9
9 2
1
7 6
6 7
1
5 8
8 5
1
10 11
11 10
1
3 1
1 3
2
4 9
9 4
2
6 12
12 6
2
8 2
2 8
2
11 7
7 11
2
1 5
5 1
2
3 10
10 3
3
6 4
4 6
3
9 8
8 9
3
12 11
11 12
3
2 1
1 2
3
7 3
3 7
3
5 10
10 5
4
4 8
8 4
4
11 6
6 11
4
1 9
9 1
4
3 12
12 3
4
10 2
2 10
4
5 7
7 5
5
11 4
4 11
5
8 1
1 8
5
6 3
3 6
5
9 10
10 9
5
12 5
5 12
5
2 7
7 2
6
4 1
1 4
6
3 11
11 3
6
10 8
8 10
6
5 6
6 5
6
7 9
9 7
6
2 12
12 2
7
3 4
4 3
7
1 10
10 1
7
11 5
5 11
7
8 7
7 8
7
6 2
2 6
7
9 12
12 9
8
4 10
10 4
8
5 3
3 5
8
7 1
1 7
8
2 11
11 2
8
12 8
8 12
8
9 6
6 9
9
5 4
4 5
9
10 7
7 10
9
3 2
2 3
9
1 12
12 1
9
11 9
9 11
9
8 6
6 8
10
4 7
7 4
10
2 5
5 2
10
12 10
10 12
10
9 3
3 9
10
6 1
1 6
10
8 11
11 8
11
2 4
4 2
11
7 12
12 7
11
5 9
9 5
11
10 6
6 10
11
3 8
8 3
11
1 11
11 1
12
4 12
12 4
12
9 2
2 9
12
6 7
7 6
12
8 5
5 8
12
11 10
10 11
12
1 3
3 1
13
9 4
4 9
13
12 6
6 12
13
2 8
8 2
13
7 11
11 7
13
5 1
1 5
13
10 3
3 10
14
4 6
6 4
14
8 9
9 8
14
11 12
12 11
14
1 2
2 1
14
3 7
7 3
14
10 5
5 10
15
8 4
4 8
15
6 11
11 6
15
9 1
1 9
15
12 3
3 1

In [ ]:
# add index on team id and matchup id